In [71]:
bounding_boxes = []
embeddings_list = []
lables = []

In [73]:
from ultralytics import YOLO
import cv2
import numpy as np
import os
import time
from keras_facenet import FaceNet

model = YOLO("/home/neeraj/Documents/face_detx/runs-20241009T055735Z-001/runs/detect/train/weights/best.pt")
embedder = FaceNet()

CONF_THRESHOLD = 0.85

name = input("Enter your name: ")
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# start_time = time.time()
count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture frame.")
        break

    results = model(source=frame)
    result = results[0]
    
    for i, box in enumerate(result.boxes):
        confidence = box.conf[0]

        if confidence >= CONF_THRESHOLD:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = round(x1.item()), round(y1.item()), round(x2.item()), round(y2.item())

            # print(f"Detected Bounding Box {i+1}: x1={x1}, y1={y1}, x2={x2}, y2={y2}")

            h, w, _ = frame.shape
            x1, y1 = max(0, x1), max(0, y1)
            x2, y2 = min(w, x2), min(h, y2)

            face = frame[y1:y2, x1:x2]

            face_resized = cv2.resize(face, (160, 160))

            face_rgb = cv2.cvtColor(face_resized, cv2.COLOR_BGR2RGB)
            mean, std = face.mean(),face.std()
            face = (face_rgb-mean)/std
            face = np.expand_dims(face, axis=0)

            embedding = embedder.embeddings(face)[0]

            bounding_boxes.append((x1, y1, x2, y2))
            embeddings_list.append(embedding)
            lables.append(name)
            count = count+1
            print(f"Face detected...{count}")
            # if(count==30):
            #     break

            # cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

    cv2.imshow('YOLOv8 FacedetX', result.plot())
    if((cv2.waitKey(1) & 0xFF == ord('q')) | len(lables)%60==0):
        break

cap.release()
cv2.destroyAllWindows()
# np.savez_compressed('face_data.npz', bounding_boxes=bounding_boxes, embeddings=embeddings_list)

print("Bounding boxes and embeddings stored successfully.")


Enter your name:  bixxy



0: 480x640 1 person, 170.4ms
Speed: 2.6ms preprocess, 170.4ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Face detected...1

0: 480x640 1 person, 200.1ms
Speed: 18.4ms preprocess, 200.1ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
Face detected...2

0: 480x640 1 person, 171.2ms
Speed: 3.7ms preprocess, 171.2ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
Face detected...3

0: 480x640 1 person, 168.1ms
Speed: 3.1ms preprocess, 168.1ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 177.7ms
Speed: 3.0ms preprocess, 177.7ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 163.4ms
Speed: 3.1ms preprocess, 163.4ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
Face detected...4

0: 480x6

In [74]:
print(f"lables:{len(lables)}")
print(lables)

lables:120
['kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'kalki', 'bixxy', 'bixxy', 'bixxy', 'bixxy', 'bixxy', 'bixxy', 'bixxy', 'bixxy', 'bixxy', 'bixxy', 'bixxy', 'bixxy', 'bixxy', 'bixxy', 'bixxy', 'bixxy', 'bixxy', 'bixxy', 'bixxy', 'bixxy', 'bixxy', 'bixxy', 'bixxy', 'bixxy', 'bixxy', 'bixxy', 'bixxy', 'bixxy', 'bixxy', 'bixxy', 'bixxy', 'bixxy', 'bixxy', 'bixxy', 'bixxy', 'bixxy', 'bixxy', 'bixxy', 'bixxy', 'bixxy', 'bixxy', 'bixxy', 'bixxy', 'bixxy', 'bixxy', 'bixxy', 'bixxy', 'bixxy', 'bixxy', 'bixxy'

In [75]:
print(f"Length of each embedding: {len(embeddings_list[0])}")
print(f"Total embeddings: {len(embeddings_list)}")


Length of each embedding: 512
Total embeddings: 120


In [76]:
embeddings_list = np.asarray(embeddings_list)

In [77]:
embeddings_list.shape

(120, 512)

In [78]:
embeddings_list[0].shape

(512,)

In [112]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
labels_enc = encoder.fit_transform(lables)

In [110]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(embeddings_list,labels_enc,shuffle=True,test_size=0.1,random_state=12)

In [111]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

# Define parameter grid
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf'],
    'gamma': [0.001, 0.01, 0.1, 1]
}

# Create the SVM model
svm = SVC()

# Perform grid search to find best parameters
grid_search = GridSearchCV(svm, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Print the best parameters found
print(f"Best parameters: {grid_search.best_params_}")


Best parameters: {'C': 0.01, 'gamma': 0.001, 'kernel': 'linear'}


In [119]:
from sklearn.svm import SVC
svm_model = SVC(kernel='linear',probability=True,gamma=0.001,C=0.01)

In [120]:
svm_model.fit(X_train,y_train)

SVC(C=0.01, gamma=0.001, kernel='linear', probability=True)

In [121]:
from sklearn.metrics import accuracy_score
y_pred = svm_model.predict(X_test)
accuracy_score(y_test,y_pred)

0.4166666666666667

In [122]:
from ultralytics import YOLO
import cv2
import numpy as np
import os
import time
from keras_facenet import FaceNet

# Load the YOLO model and FaceNet
model = YOLO("/home/neeraj/Documents/face_detx/runs-20241009T055735Z-001/runs/detect/train/weights/best.pt")
embedder = FaceNet()

CONF_THRESHOLD = 0.85  # Confidence threshold for detection

# Initialize variables
name = input("Enter your name: ")
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

bounding_box = []
embedding_lst = []
label = []

count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture frame.")
        break

    # Display the live camera feed
    cv2.imshow('YOLOv8 FacedetX', frame)

    # Wait for keypress (Spacebar to take a picture, 'q' to quit)
    key = cv2.waitKey(1) & 0xFF
    if key == ord(' '):  # Spacebar pressed to take the picture
        print("Processing frame...")

        # Run YOLO model on the captured frame
        results = model(source=frame)
        result = results[0]

        for i, box in enumerate(result.boxes):
            confidence = box.conf[0]

            if confidence >= CONF_THRESHOLD:
                # Get bounding box coordinates
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = round(x1.item()), round(y1.item()), round(x2.item()), round(y2.item())

                # Ensure bounding box is within the frame
                h, w, _ = frame.shape
                x1, y1 = max(0, x1), max(0, y1)
                x2, y2 = min(w, x2), min(h, y2)

                # Crop and preprocess the face
                face = frame[y1:y2, x1:x2]
                face_resized = cv2.resize(face, (160, 160))
                face_rgb = cv2.cvtColor(face_resized, cv2.COLOR_BGR2RGB)

                # Normalize the face image
                mean, std = face_rgb.mean(), face_rgb.std()
                face = (face_rgb - mean) / std
                face = np.expand_dims(face, axis=0)

                # Generate embeddings
                embedding = embedder.embeddings(face)[0]

                # Store bounding box, embedding, and label
                bounding_box.append((x1, y1, x2, y2))
                
                embedding_lst.append(embedding)
                label.append(name)

                count += 1
                print(f"Face detected and saved... Count: {count}")

                # Draw the bounding box on the frame (for visualization)
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Show the frame with the bounding box
        cv2.imshow('YOLOv8 FacedetX', result.plot())

    elif key == ord('q'):  # Quit if 'q' is pressed or if labels length is a multiple of 30
        print("Exiting...")
        break

# Release the camera and close all windows
cap.release()
cv2.destroyAllWindows()

# Save the embeddings and bounding boxes to a file (if needed)
# np.savez_compressed('face_data.npz', bounding_boxes=bounding_boxes, embeddings=embeddings_list)

print("Bounding boxes and embeddings stored successfully.")


Enter your name:  Neeraj


Processing frame...

0: 480x640 1 person, 170.9ms
Speed: 2.0ms preprocess, 170.9ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Face detected and saved... Count: 1
Exiting...
Bounding boxes and embeddings stored successfully.


In [128]:
embedding = np.asarray(embedding_lst[0]).flatten()
embedding_reshaped = embedding.reshape(1, -1)
pred = svm_model.predict(embedding_reshaped)
name = encoder.inverse_transform(pred)[0]
print(pred)
print(name)

[0]
bixxy


For incremental learning

In [ ]:
new_embeddings = []
new_labels = []

In [81]:
from ultralytics import YOLO
import cv2
import numpy as np
import os
import time
from keras_facenet import FaceNet

model = YOLO("/home/neeraj/Documents/face_detx/runs-20241009T055735Z-001/runs/detect/train/weights/best.pt")
embedder = FaceNet()

CONF_THRESHOLD = 0.85

name = input("Enter your name: ")
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# start_time = time.time()
count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture frame.")
        break

    results = model(source=frame)
    result = results[0]
    
    for i, box in enumerate(result.boxes):
        confidence = box.conf[0]

        if confidence >= CONF_THRESHOLD:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = round(x1.item()), round(y1.item()), round(x2.item()), round(y2.item())

            # print(f"Detected Bounding Box {i+1}: x1={x1}, y1={y1}, x2={x2}, y2={y2}")

            h, w, _ = frame.shape
            x1, y1 = max(0, x1), max(0, y1)
            x2, y2 = min(w, x2), min(h, y2)

            face = frame[y1:y2, x1:x2]

            face_resized = cv2.resize(face, (160, 160))

            face_rgb = cv2.cvtColor(face_resized, cv2.COLOR_BGR2RGB)
            mean, std = face.mean(),face.std()
            face = (face_rgb-mean)/std
            face = np.expand_dims(face, axis=0)

            embedding = embedder.embeddings(face)[0]

            # bounding_boxes.append((x1, y1, x2, y2))
            new_embeddings.append(embedding)
            new_labels.append(name)
            count = count+1
            print(f"Face detected...{count}")
            # if(count==30):
            #     break

            # cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

    cv2.imshow('YOLOv8 FacedetX', result.plot())
    if((cv2.waitKey(1) & 0xFF == ord('q')) | len(lables)%30==0):
        break

cap.release()
cv2.destroyAllWindows()
# np.savez_compressed('face_data.npz', bounding_boxes=bounding_boxes, embeddings=embeddings_list)

print("Bounding boxes and embeddings stored successfully.")


Enter your name:  Jerry



0: 480x640 1 person, 218.9ms
Speed: 6.4ms preprocess, 218.9ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
Face detected...1
Bounding boxes and embeddings stored successfully.


In [ ]:
new_labels_enc = encoder.fit_transform(new_labels)

In [ ]:
new_embeddings = np.asarray(new_embeddings)

In [ ]:
svm.partial_fit(new_embeddings, new_labels_enc, classes=np.unique(labels_enc))